In [ ]:
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 250)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../data/sites_ABC.csv')
df.columns = ['site','timestamp','temp','irr','power','demand_response','demand_response_capacity']

non_feature_cols = ['site','timestamp','demand_response','demand_response_capacity','date','busday','time','minute']
base_features = ['temp','irr','power']
target_cls = 'demand_response'
target_reg = 'demand_response_capacity'
working_hours = [10,11,12,13,14,15,16,17]
relevant_hours = [8,9,10,11,12,13,14,15,16,17]

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['busday'] = np.is_busday(df['timestamp'].to_numpy().astype("datetime64[D]")).astype(int)
df['date'] = pd.to_datetime(df['timestamp'].dt.date)
df['day_of_week'] = df['date'].dt.weekday
df['week'] = df['timestamp'].dt.isocalendar().week.astype(int)
df['hour'] = df['timestamp'].dt.hour
df['time'] = df['timestamp'].dt.time
df['quarter_hour'] = df['timestamp'].dt.minute//15
df['month'] = df['timestamp'].dt.month
df['season'] = 0
df.loc[df['month'].isin([12,1,2]), 'season'] = 0
df.loc[df['month'].isin([3,4,5]), 'season'] = 1
df.loc[df['month'].isin([6,7,8]), 'season'] = 2
df.loc[df['month'].isin([9,10,11]), 'season'] = 3
df['working_hours'] = 0
df.loc[df['hour'].isin(working_hours), 'working_hours'] = 1

In [ ]:
def interpolate_prediction(group):
    dr_nonzero = group['li_flag'] != 0
    nonzero_indices = group.index[dr_nonzero]

    if nonzero_indices.empty:
        group['li_feature'] = np.nan
        return group

    start_idx = nonzero_indices[0]
    end_idx = nonzero_indices[-1]

    before_idx = start_idx - 1
    after_idx = end_idx + 1

    if before_idx < group.index[0] or after_idx > group.index[-1]:
        group['li_feature'] = np.nan
        return group

    start_power = group.loc[before_idx, 'power']
    end_power = group.loc[after_idx, 'power']

    num_points = end_idx - start_idx + 1
    interpolated = np.linspace(start_power, end_power, num_points)

    group['li_feature'] = np.nan
    group.loc[start_idx:end_idx, 'li_feature'] = interpolated

    return group

def fill_between_activity(df):
    df = df.copy()

    def fill_day(g):
        # find first and last hour where pow_above_base == 1
        if g['pow_above_base'].sum() == 0:
            return g  # no active hours, do nothing
        first_on = g.loc[g['pow_above_base'] == 1, 'time'].min()
        last_on  = g.loc[g['pow_above_base'] == 1, 'time'].max()

        # fill all hours between those two with 1
        g.loc[(g['time'] >= first_on) & (g['time'] <= last_on), 'pow_above_base'] = 1
        return g

    df = df.groupby(['site', 'date'], group_keys=False).apply(fill_day)
    return df

def feature_engineering(og_df):
    df = og_df.copy()
    
    baseline_partition_columns = ['site', 'date']
    baseline_hours = [5,6,7]
    baseline_powers = df[df['hour'].isin(baseline_hours)].groupby(baseline_partition_columns)['power'].median().to_dict()
    df['baseline_pow'] = (
        df[baseline_partition_columns]
        .apply(tuple, axis=1)
        .map(baseline_powers)
        * 1.2
    )
    df['pow_above_base'] = (df['power'] > df['baseline_pow']).astype(int)
    df = fill_between_activity(df)
    df['site_on'] = df.groupby(['site','date'])['pow_above_base'].cumsum()
    df.loc[df['pow_above_base'] == 0, 'site_on'] = 0
    
    import datetime as dt
    df['li_flag'] = df['time'].apply(lambda t: int(dt.time(10,0) <= t <= dt.time(18,0)))
    df = df.groupby(['site', 'date'], group_keys=False).apply(interpolate_prediction)

    # df = df[df['working_hours'] == 1].reset_index(drop=True).copy()
    for var in ['temp', 'irr']:
        # per-site, month, time correlation of power with var
        corr_map = df.groupby(['site','month','day_of_week','time']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_mt')

        df = df.merge(corr_map, on=['site','month','day_of_week','time'], how='left')

        corr_map = df.groupby(['site','season','day_of_week','time']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_st')

        df = df.merge(corr_map, on=['site','season','day_of_week','time'], how='left')

        # per-site, month, time correlation of power with var POW ABOVE BASE ONLY
        corr_map = df.loc[df['pow_above_base'] == 1].groupby(['site','month','day_of_week','time']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_mt_pab')

        df = df.merge(corr_map, on=['site','month','day_of_week','time'], how='left')

        corr_map = df.loc[df['pow_above_base'] == 1].groupby(['site','season','day_of_week','time']).apply(
            lambda g: g['power'].corr(g[var])
        ).rename(f'{var}_power_corr_st_pab')

        df = df.merge(corr_map, on=['site','season','day_of_week','time'], how='left')

        # median of var per site, month, time
        med_map = df.groupby(['site','month','time'])[var].median().rename(f'{var}_median_mt')
        df = df.merge(med_map, on=['site','month','time'], how='left')

        med_map = df.groupby(['site','season','time'])[var].median().rename(f'{var}_median_st')
        df = df.merge(med_map, on=['site','season','time'], how='left')

        # interaction term
        df[f'{var}_corr_dev_mt'] = df[f'{var}_power_corr_mt'] * (df[var] - df[f'{var}_median_mt'])
        df[f'{var}_corr_dev_st'] = df[f'{var}_power_corr_st'] * (df[var] - df[f'{var}_median_st'])

        # interaction term pab
        df[f'{var}_corr_dev_mt_pab'] = df[f'{var}_power_corr_mt_pab'] * (df[var] - df[f'{var}_median_mt'])
        df[f'{var}_corr_dev_st_pab'] = df[f'{var}_power_corr_st_pab'] * (df[var] - df[f'{var}_median_st'])

    df.fillna(0, inplace=True)

    for diff in [1,2,4,12]:
        df[f"temp_diff{diff if diff > 1 else ''}"] = df.groupby(['site'])['temp'].diff(diff).fillna(0)
        df[f"irr_diff{diff if diff > 1 else ''}"] = df.groupby(['site'])['irr'].diff(diff).fillna(0)

    features = [
        'temp', 'irr', 
        'temp_diff', 'irr_diff',
        'temp_diff2', 'irr_diff2',
        'temp_diff4', 'irr_diff4',
        'temp_diff12', 'irr_diff12'
    ]
    group_levels = [['site', 'time'], ['site']]
    suffixes = {
        ('site', 'time'): ('lag', 'peek'),
        ('site',): ('shift', 'pull')
    }
    shifts = [1, 2, 3, 4, 8, 12]

    for feature in features:
        for group in group_levels:
            suffix_pair = suffixes[tuple(group)]
            for direction, suffix in zip([1, -1], suffix_pair):
                for s in shifts:
                    col_name = f"{feature}_{suffix}{s if s > 1 else ''}"
                    df[col_name] = df.groupby(group)[feature].transform(lambda x, shift=s*direction: x.shift(shift))

    df['usage_lag'] = df.groupby(['site','time'])['power'].transform(lambda x: x.shift(1))
    df['usage_lag2'] = df.groupby(['site','time'])['power'].transform(lambda x: x.shift(2))
    df['usage_lag_dow'] = df.groupby(['site','day_of_week','time'])['power'].transform(lambda x: x.shift(1))
    df['usage_lag_dow2'] = df.groupby(['site','day_of_week','time'])['power'].transform(lambda x: x.shift(2))

    df['usage_peek'] = df.groupby(['site','time'])['power'].transform(lambda x: x.shift(-1))
    df['usage_peek2'] = df.groupby(['site','time'])['power'].transform(lambda x: x.shift(-2))
    df['usage_peek_dow'] = df.groupby(['site','day_of_week','time'])['power'].transform(lambda x: x.shift(-1))
    df['usage_peek_dow2'] = df.groupby(['site','day_of_week','time'])['power'].transform(lambda x: x.shift(-2))

    df['mean_usage_mdt'] = df.groupby(['site','month','day_of_week','time'])['power'].transform(lambda x: x.mean())
    df['mean_usage_sdt'] = df.groupby(['site','season','day_of_week','time'])['power'].transform(lambda x: x.mean())

    df['mean_usage_mdt_corr_dev_mt'] = (df['mean_usage_mdt'] + df['temp_corr_dev_mt']*1.5 + df['irr_corr_dev_mt'] * 0.05)
    df['mean_usage_sdt_corr_dev_st'] = (df['mean_usage_sdt'] + df['temp_corr_dev_st']*1.5 + df['irr_corr_dev_st'] * 0.05)

    df['mean_usage_mdt_corr_dev_mt_pab'] = (df['mean_usage_mdt'] + df['temp_corr_dev_mt_pab']*1.5 + df['irr_corr_dev_mt_pab'] * 0.05)
    df['mean_usage_sdt_corr_dev_st_pab'] = (df['mean_usage_sdt'] + df['temp_corr_dev_st_pab']*1.5 + df['irr_corr_dev_st_pab'] * 0.05)

    df['pow_monthly_max'] = df.groupby(['site','month'])['power'].transform('max')

    return df.fillna(0)

In [ ]:
df['day_response']     = 0 # -1,0,1,2

positive_flag_dates = df.groupby(["site", "date"]).filter(
    lambda df: (df["demand_response"] == 1).any() and not (df["demand_response"] == -1).any()
)[["site", "date"]].drop_duplicates()

negative_flag_dates = df.groupby(["site", "date"]).filter(
    lambda df: (df["demand_response"] == -1).any() and not (df["demand_response"] == 1).any()
)[["site", "date"]].drop_duplicates()

pos_and_neg_flag_dates = df.groupby(["site", "date"]).filter(
    lambda df: (df["demand_response"] == 1).any() and (df["demand_response"] == -1).any()
)[["site", "date"]].drop_duplicates()

# Create a tuple column for easier comparison
df["site_date"] = list(zip(df["site"], df["date"]))

# Create sets of tuples for comparison
neg_set = set(zip(negative_flag_dates["site"], negative_flag_dates["date"]))
pos_set = set(zip(positive_flag_dates["site"], positive_flag_dates["date"]))
both_set = set(zip(pos_and_neg_flag_dates["site"], pos_and_neg_flag_dates["date"]))

# Assign values
df.loc[df["site_date"].isin(neg_set), "day_response"] = -1
df.loc[df["site_date"].isin(pos_set), "day_response"] = 1
df.loc[df["site_date"].isin(both_set), "day_response"] = 2

# Optionally drop the helper column
df.drop(columns="site_date", inplace=True)

In [ ]:
df_wh = feature_engineering(df)

In [ ]:
df_no_response = df_wh[(df_wh['day_response'] == 0) & (df_wh['working_hours'] == 1)].copy()
print(len(df_no_response)//(10*4))
df_no_response.head()

In [ ]:
df_response = df_wh[(df_wh['day_response'] != 0) & (df_wh['working_hours'] == 1)].copy()
print(len(df_response)//(10*4))
df_response.head()

In [ ]:
def rmse(y, y_hat, return_median=False):
  if not return_median:
    return np.sqrt(np.mean((y - y_hat) ** 2))
  else:
    return np.sqrt(np.median((y - y_hat) ** 2))

def r_squared(y, y_hat):
  ssr = np.sum((y - y_hat) ** 2)
  sst = np.sum((y - np.mean(y)) ** 2)
  return 1 - (ssr / sst)

def mape(y, y_hat, return_median=False):
  if not return_median:
    return np.mean(np.abs((y - y_hat) / y))*100
  else:
    return np.median(np.abs((y - y_hat) / y))*100

def mpe(y, y_hat, return_median=False): # Negative means over-prediction
  if not return_median:
    return np.mean((y - y_hat) / y)*100
  else:
    return np.median((y - y_hat) / y)*100

def lw_mape(y, y_hat, add_epsilon=True):
  if add_epsilon: # Avoids division by zero (if needed)
    epsilon = 1e-8
    y += epsilon
  w = np.abs(y) / np.abs(y).sum()
  return np.sum(np.abs((y - y_hat) / y) * w)*100

def evaluate_abc_partitioned_site(df, pred_col='capacity_prediction'):
  res = []
  for site in ['A', 'B', 'C']:
        ixs = (df['site']==f'site{site}')&(df['demand_response']!=0)
        res.append({
        "site":site,
        "RMSE":rmse(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "MAE":mean_absolute_error(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "W_MAPE (%)":lw_mape(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "Num_Days":len(df.loc[ixs, 'date'].drop_duplicates()),
        })
  return pd.DataFrame(res).set_index('site')

def evaluate_abc_partitioned_site_dayresponse(df, pred_col='capacity_prediction'):
  res = []
  for site in ['A', 'B', 'C']:
    for day_response in [-1,1,2]:
        ixs = (df['site']==f'site{site}')&(df['day_response']==day_response)&(df['demand_response']!=0)
        res.append({
        "site":site,
        "day_response":day_response,
        "RMSE":rmse(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "MAE":mean_absolute_error(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "W_MAPE (%)":lw_mape(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "Num_Days":len(df.loc[ixs, 'date'].drop_duplicates()),
        })
  return pd.DataFrame(res).set_index(['site', 'day_response'])

def evaluate_abc_partitioned_site_response(df, pred_col='capacity_prediction'):
  res = []
  for site in ['A', 'B', 'C']:
    for demand_response in [-1,1]:
        ixs = (df['site']==f'site{site}')&(df['demand_response']==demand_response)&(df['demand_response']!=0)
        res.append({
        "site":site,
        "demand_response":demand_response,
        "RMSE":rmse(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "MAE":mean_absolute_error(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "W_MAPE (%)":lw_mape(df.loc[ixs, 'demand_response_capacity'], df.loc[ixs, pred_col]),
        "Num_Days":len(df.loc[ixs, 'date'].drop_duplicates()),
        })
        
  return pd.DataFrame(res).set_index(['site', 'demand_response'])

In [ ]:
feats = ['day_of_week', 'month', 'li_feature', 'temp_lag', 'temp_lag3', 'temp_lag4', 'temp_peek', 'irr_lag', 'irr_lag2', 'irr_lag3', 'irr_lag4', 'irr_peek2', 'irr_peek3', 'irr_peek4', 'temp_shift4', 'irr_shift', 'irr_shift2', 'irr_shift3', 'irr_shift4', 'irr_pull', 'irr_pull2', 'irr_pull4', 'usage_lag_dow2', 'usage_lag_dow', 'hour', 'irr_pull3', 'temp_median_mt', 'irr_median_st', 'week', 'temp_pull3', 'irr_power_corr_st', 'irr_peek', 'baseline_pow', 'quarter_hour', 'temp_pull2', 'temp_lag2'] 
feats2 = ['irr_lag', 'irr_peek3', 'temp_lag', 'temp_peek2', 'season', 'month', 'day_of_week', 'hour', 'temp', 'irr', 'mean_usage_sdt_corr_dev_st', 'usage_lag', 'usage_lag_dow', 'usage_peek', 'usage_peek_dow', 'temp_shift3', 'temp_lag3', 'li_feature', 'baseline_pow', 'temp_pull3', 'temp_shift12', 'irr_diff4_shift3'] 
feats3 = ['day_of_week', 'hour', 'month', 'mean_usage_sdt_corr_dev_st', 'li_feature', 'temp_lag', 'temp_shift4', 'week', 'irr_power_corr_st_pab', 'mean_usage_sdt', 'baseline_pow', 'usage_peek2', 'irr_median_mt', 'irr_power_corr_st', 'usage_lag', 'temp_shift', 'temp_power_corr_st', 'irr_shift4', 'temp_lag3', 'irr_pull2', ] 
feats4 = ['day_of_week', 'hour', 'month', 'li_feature', 'temp_shift4', 'temp_lag', 'irr_diff2_peek2', 'week', 'irr_diff12_pull', 'temp_median_st', 'mean_usage_sdt_corr_dev_st_pab', 'temp_diff12_shift4', 'irr_diff4_pull3', 'usage_lag_dow', 'temp_lag2'] 
feats5 = ['day_of_week', 'hour', 'month', 'li_feature', 'temp_shift12', 'week', 'temp_lag', 'mean_usage_sdt_corr_dev_st_pab', 'irr_diff4_shift4', 'baseline_pow', 'irr_diff_peek12', 'irr_diff4_shift2', 'pow_monthly_max', 'irr_diff4_lag8', 'irr_diff12_lag12', 'temp_shift', 'usage_lag_dow2', 'irr_diff2_pull8'] 

In [ ]:
# Define models and their feature sets
models = {
    'xgbr': (xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.05778072586944159,subsample=0.6616312346972526,max_depth=9,colsample_bytree=0.6352137434228233, n_estimators=184), feats),
    'xgbr2': (xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.05513689802291822,subsample=0.5167731642508813,max_depth=9,colsample_bytree=0.7395754768661291, n_estimators=104), feats2),
    'xgbr3': (xgb.XGBRegressor(objective='reg:absoluteerror',random_state=42,eta=0.06449597248839985,subsample=0.9767027104796806,max_depth=8,colsample_bytree=0.450212514664335, n_estimators=197), feats3),
    'xgbr4': (xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.0636975789822504,subsample=0.6608736639449571,max_depth=9,colsample_bytree=0.6033560282663871, n_estimators=162), feats4),
    'xgbr5': (xgb.XGBRegressor(objective='reg:absoluteerror',random_state=42,eta=0.06449597248839985,subsample=0.9767027104796806,max_depth=8,colsample_bytree=0.450212514664335, n_estimators=300), feats5),
}

# Initialize prediction columns
for model_name in models:
    df_response[f'power_prediction_{model_name}'] = 0.0
    df_response[f'capacity_prediction_{model_name}'] = 0.0

# Train and predict per site
for site in ['A', 'B', 'C']:
    site_label = f'site{site}'

    sub_no_response = df_no_response[df_no_response['site'] == site_label].copy()
    sub_response = df_response[df_response['site'] == site_label]      

    y = sub_no_response['power']

    for model_name, (model, features) in models.items():
        X_train = sub_no_response[features]
        X_test = sub_response[features]

        model.fit(X_train, y)
        predictions = model.predict(X_test)

        # Store predictions
        df_response.loc[df_response['site'] == site_label, f'power_prediction_{model_name}'] = predictions

# Zero out predictions where demand_response == 0
for model_name in models:
    mask = df_response['demand_response'] == 0
    df_response.loc[mask, f'power_prediction_{model_name}'] = 0.0
# 
# Compute capacity predictions where demand_response != 0
for model_name in models:
    mask = df_response['demand_response'] != 0
    df_response.loc[mask, f'capacity_prediction_{model_name}'] = (
        df_response.loc[mask, 'power'] - df_response.loc[mask, f'power_prediction_{model_name}']
    )

In [ ]:
# df_response['full_capacity_prediction'] = df_response['capacity_prediction_xgbr'] * 0.45 + df_response['capacity_prediction_xgbr2'] * 0.3 + df_response['capacity_prediction_xgbr3'] * 0.15 + df_response['capacity_prediction_xgbr4'] * 0.1
df_response['full_capacity_prediction'] = df_response['capacity_prediction_xgbr'] * 0.45 + df_response['capacity_prediction_xgbr2'] * 0.3 + df_response['capacity_prediction_xgbr3'] * 0.15 + df_response['capacity_prediction_xgbr4'] * 0.1
df_response.loc[df_response['demand_response'] == -1, 'full_capacity_prediction'] = df_response.loc[df_response['demand_response'] == -1]['capacity_prediction_xgbr'] * 0.1 + df_response.loc[df_response['demand_response'] == -1]['capacity_prediction_xgbr2'] * 0.1 + df_response.loc[df_response['demand_response'] == -1]['capacity_prediction_xgbr3'] * 0.1 + df_response.loc[df_response['demand_response'] == -1]['capacity_prediction_xgbr4'] * 0.1 + df_response.loc[df_response['demand_response'] == -1]['capacity_prediction_xgbr5'] * 0.6


In [ ]:
pred_cols = [col for col in df_response.columns if 'full_capacity_prediction' in col and 'clipped' not in col]
# pred_cols = [col for col in df_response.columns if 'capacity_prediction' in col and 'full' not in col]

for pred_col in pred_cols:
    print(pred_col)
    print(evaluate_abc_partitioned_site(df_response, pred_col=pred_col).round(2), '\n')

In [ ]:
for pred_col in pred_cols:
    print(pred_col)
    print(evaluate_abc_partitioned_site_dayresponse(df_response, pred_col=pred_col).round(2), '\n')

In [ ]:
for pred_col in pred_cols:
    print(pred_col)
    print(evaluate_abc_partitioned_site_response(df_response, pred_col=pred_col).round(2), '\n')

In [ ]:
small_site_neg_clips = (-.25, .225)
small_site_pos_clips = (0, .4)
large_site_neg_clips = (-.2, .125)
large_site_pos_clips = (0.0, .3)

for site in ['A', 'B', 'C']:
    site_label = f'site{site}'
    max_pow = df_wh[(df_wh['site'] == site_label) & (df_wh['working_hours'] == 1)]['power'].max()
    if max_pow < 100:
        # neg clips
        df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1)]['full_capacity_prediction']*1.04, small_site_neg_clips[0]*max_pow, small_site_neg_clips[1]*max_pow)
        # pos clips
        df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1)]['full_capacity_prediction']*1.03, small_site_pos_clips[0]*max_pow, small_site_pos_clips[1]*max_pow)
        # neg clips
        # df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1)]['full_capacity_prediction'], small_site_neg_clips[0]*max_pow, small_site_neg_clips[1]*max_pow)
        # pos clips
        # df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1)]['full_capacity_prediction'], small_site_pos_clips[0]*max_pow, small_site_pos_clips[1]*max_pow)
    else:
        # neg clips
        df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1)]['full_capacity_prediction']*1.04, large_site_neg_clips[0]*max_pow, large_site_neg_clips[1]*max_pow)
        # pos clips
        df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1)]['full_capacity_prediction']*1.03, large_site_pos_clips[0]*max_pow, large_site_pos_clips[1]*max_pow)
        # neg clips
        # df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == -1)]['full_capacity_prediction'], large_site_neg_clips[0]*max_pow, large_site_neg_clips[1]*max_pow)
        # pos clips
        # df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1), 'full_capacity_prediction_clipped'] = np.clip(df_response.loc[(df_response['site'] == site_label) & (df_response['demand_response'] == 1)]['full_capacity_prediction'], large_site_pos_clips[0]*max_pow, large_site_pos_clips[1]*max_pow)

In [ ]:
pred_cols = [col for col in df_response.columns if 'full_capacity_prediction' in col]
# pred_cols = [col for col in df_response.columns if 'capacity_prediction' in col]

for pred_col in pred_cols:
    print(pred_col)
    print(evaluate_abc_partitioned_site(df_response, pred_col=pred_col).round(2), '\n')

In [ ]:
# full_capacity_prediction_clipped
#        RMSE    MAE  W_MAPE (%)  Num_Days
# site                                    
# A      1.17   0.81       39.77        28
# B      1.46   1.06       31.19        41
# C     21.05  12.84       47.21        46 

In [ ]:
import optuna

# ===============================
# Objective function
# ===============================
def objective(trial, model_name, df_no_response, df_response, features):
    scores = []

    for site in ['A', 'B', 'C']:
        site_label = f'site{site}'
        sub_no_response = df_no_response[df_no_response['site'] == site_label].copy()
        sub_response = df_response[df_response['site'] == site_label].copy()

        y = sub_no_response['power']

        # Choose model + hyperparams
        if model_name == "xgbr":
            params = {
                "objective": "reg:absoluteerror",
                "random_state": 42,
                # "eta": trial.suggest_float("eta", 0.04, 0.1),
                # "eta": trial.suggest_categorical("eta", [0.07263299854682012, 0.06335869128027055]),
                "eta": 0.06449597248839985,
                # "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "subsample": 0.9767027104796806,
                # "max_depth": trial.suggest_int("max_depth", 8, 12),
                "max_depth": 8,
                # "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
                "colsample_bytree": 0.450212514664335,
                "n_estimators": trial.suggest_int("n_estimators", 250, 400)
            }
            model = xgb.XGBRegressor(**params)

        # Train model
        X_train = sub_no_response[features]
        X_test = sub_response[features]
        model.fit(X_train, y)
        predictions = model.predict(X_test)

        # Zero-out demand_response == 0
        sub_response["power_pred"] = predictions
        sub_response.loc[sub_response['demand_response'] == 0, "power_pred"] = 0.0

        # Capacity prediction
        sub_response["capacity_pred"] = 0.0
        mask = sub_response['demand_response'] != 0
        sub_response.loc[mask, "capacity_pred"] = (
            sub_response.loc[mask, "power"] - sub_response.loc[mask, "power_pred"]
        )

        power_mean = pd.concat([
            sub_no_response.loc[sub_no_response['power'] != 0, 'power'],
            sub_response.loc[sub_response['power'] != 0, 'power']
        ]).mean()

        mae = mean_absolute_error(
            sub_response.loc[mask, "demand_response_capacity"],
            sub_response.loc[mask, "capacity_pred"]
        )

        n_mae_mean = (mae / power_mean) * 100
        scores.append(n_mae_mean)
        
    return np.mean(scores)

# ===============================
# Run study
# ===============================
def run_optuna(model_name, df_no_response, df_response, features, n_trials=50):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, model_name, df_no_response, df_response, features),
                   n_trials=n_trials)
    print(f"Best trial for {model_name}: {study.best_trial.params}")
    return study


In [ ]:
# {'eta': 0.05778072586944159, 'subsample': 0.6616312346972526, 'max_depth': 9, 'colsample_bytree': 0.6352137434228233} 10.321465712910529
# {'eta': 0.05513689802291822, 'subsample': 0.5167731642508813, 'colsample_bytree': 0.7395754768661291} 10.751949817155186
# {'eta': 0.06449597248839985, 'subsample': 0.9767027104796806, 'max_depth': 8, 'colsample_bytree': 0.450212514664335} 10.488023979638237 || 10.4
# {'eta': 0.06335869128027055, 'subsample': 0.8518532235993314, 'max_depth': 9, 'colsample_bytree': 0.7880229765180591} 10.50
study_xgb = run_optuna("xgbr", df_no_response, df_response, selected_features, n_trials=50) # 10.188453141953326
# study_lgb = run_optuna("lgbr", df_no_response, df_response, selected_features, n_trials=100)
# study_cat = run_optuna("cat", df_no_response, df_response, n_trials=100)

In [ ]:
study_xgb.trials_dataframe().drop_duplicates(subset=[col for col in study_xgb.trials_dataframe().columns if 'params_' in col]).sort_values(by='value').head(5)

In [ ]:
# study_xgb.trials_dataframe().sort_values(by='value').head(10).reset_index(drop=True) # 5.767547
# study_lgb.trials_dataframe().sort_values(by='value').head(10).reset_index(drop=True)
# study_cat.trials_dataframe().sort_values(by='value').head(10).reset_index(drop=True).to_csv('cat_optuna.csv')

In [ ]:
from tqdm import tqdm

def feature_selection(df_no_response, df_response, model, starting_features, features, target='demand_response_capacity'):
    results = []

    # Baseline with starting features
    baseline_score = evaluate_features(df_no_response, df_response, starting_features, target=target, model=model)
    print(f"Baseline score: {baseline_score:.4f}")

    for feat in tqdm(features, desc="Testing features", unit="feat"):
        if feat not in starting_features:
            test_feats = starting_features + [feat]
            # print(f"Testing feature: {feat}")

            score = evaluate_features(df_no_response, df_response, test_feats, target=target, model=model)
            improvement = score - baseline_score

            results.append({
                "feature": feat,
                "score": score,
                "improvement": improvement
            })

    results_df = pd.DataFrame(results).sort_values("improvement")
    return results_df

def feature_ablation(df_no_response, df_response, model, feature_list, dnt_list, target='demand_response_capacity'):
    results = []

    # Baseline with all features
    baseline_score = evaluate_features(df_no_response, df_response, feature_list, target=target, model=model)
    print(f"Baseline score: {baseline_score:.4f}")

    features_to_test = [f for f in feature_list if f not in dnt_list]
    # Iterate by removing each feature once
    for feat in tqdm(features_to_test, desc="Ablating features", unit="feat"):
        reduced_feats = [f for f in feature_list if f != feat]
        score = evaluate_features(df_no_response, df_response, reduced_feats, target=target, model=model)
        change = score - baseline_score

        results.append({
            "feature": feat,
            "score": score,
            "change_vs_full": change
        })

    results_df = pd.DataFrame(results).sort_values("change_vs_full")
    return results_df

def evaluate_features(df_no_response, df_response, feature_list, target="demand_response_capacity", model=None):
    """
    Parameters
    ----------
    df_no_response : pd.DataFrame
        Subset of data where demand_response == 0 (training data).
    df_response : pd.DataFrame
        Subset of data where demand_response != 0 (prediction/evaluation data).
    feature_list : list
        Features to use.
    target : str
        Target column to evaluate against (default: demand_response_capacity).
    model : object
        Any sklearn-style model with .fit() and .predict().
    """
    if model is None:
        raise ValueError("You must pass a model instance with .fit() and .predict() methods.")

    df_eval = df_response.copy()
    df_eval["capacity_pred"] = 0.0

    for site in df_no_response["site"].unique():
        site_label = site
        sub_no_response = df_no_response[df_no_response["site"] == site_label].copy()
        sub_response = df_eval[df_eval["site"] == site_label].copy()

        y_train = sub_no_response["power"]
        X_train = sub_no_response[feature_list]
        X_test = sub_response[feature_list]

        # Train model
        model.fit(X_train, y_train)

        # Predict power
        preds = model.predict(X_test)

        # Zero-out where demand_response == 0
        sub_response["power_pred"] = 0.0
        mask = sub_response["demand_response"] != 0
        sub_response.loc[mask, "power_pred"] = preds[mask]

        # Compute capacity predictions
        sub_response["capacity_pred"] = 0.0
        sub_response.loc[mask, "capacity_pred"] = (
            sub_response.loc[mask, "power"] - sub_response.loc[mask, "power_pred"]
        )

        # Update df_eval
        df_eval.loc[df_eval["site"] == site_label, "capacity_pred"] = sub_response["capacity_pred"].values

    nmaes = []
    for site in df_no_response["site"].unique():
        power_mean = pd.concat([df_no_response.loc[(df_no_response['site'] == site) & (df_no_response['power'] != 0)]['power'], df_eval.loc[(df_eval['site'] == site) & (df_eval['power'] != 0)]['power']]).mean()
        mae = mean_absolute_error(df_eval.loc[(df_eval['site'] == site) & (df_eval["demand_response"] != 0), target], df_eval.loc[(df_eval['site'] == site) & (df_eval["demand_response"] != 0), "capacity_pred"])
        n_mae_mean = (mae / power_mean)
        response_dates = len(df_eval['date'].drop_duplicates())
        nmaes.append(n_mae_mean * response_dates)

    # Evaluate on demand_response != 0 rows
    return round(np.mean(nmaes), 4)

In [ ]:
all_features = [col for col in df_wh.columns if col not in non_feature_cols + ['demand_response','demand_response_capacity','power','day_response','li_flag','working_hours'] + [col for col in df_wh.columns if 'residual' in col]]
print(len(all_features))

In [ ]:
[col for col in all_features if col in feats and col in feats2 and col in feats3 and col in feats4]

In [ ]:
# test_model = xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.05778072586944159,subsample=0.6616312346972526,max_depth=9,colsample_bytree=0.6352137434228233, n_estimators=184)
# test_model = xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.05513689802291822,subsample=0.5167731642508813,max_depth=9,colsample_bytree=0.7395754768661291, n_estimators=104)
# test_model = xgb.XGBRegressor(objective='reg:absoluteerror',random_state=42,eta=0.06449597248839985,subsample=0.9767027104796806,max_depth=8,colsample_bytree=0.450212514664335, n_estimators=197)
# test_model = xgb.XGBRegressor(objective='reg:squarederror',random_state=42,eta=0.0636975789822504,subsample=0.6608736639449571,max_depth=9,colsample_bytree=0.6033560282663871, n_estimators=162)
# test_model = xgb.XGBRegressor(objective='reg:absoluteerror',random_state=42,eta=0.06449597248839985,subsample=0.9767027104796806,max_depth=8,colsample_bytree=0.450212514664335, n_estimators=300)


In [ ]:
# base_features = ['day_of_week','hour','month','li_feature'] + [col for col in all_features if any(item in col for item in ['temp','irr'])]
base_features = ['day_of_week','hour','month','li_feature']
print(len(base_features))
print(base_features)

In [ ]:
selected_features = list(set(feats+feats2+feats3+feats4)).copy()
# selected_features = feats4.copy()
# selected_features = base_features.copy()
print(len(selected_features))

In [ ]:
selected_features = ['day_of_week', 'hour', 'month', 'li_feature', 'temp_shift12', 'week', 'temp_lag', 'mean_usage_sdt_corr_dev_st_pab', 'irr_diff4_shift4', 'baseline_pow', 'irr_diff_peek12', 'irr_diff4_shift2', 'pow_monthly_max', 'irr_diff4_lag8', 'irr_diff12_lag12', 'temp_shift', 'usage_lag_dow2', 'irr_diff2_pull8'] 
print(len(selected_features))

In [ ]:
# best scores 10.362
baseline_score = evaluate_features(df_no_response, df_response, selected_features, target=target_reg, model=test_model)
print(baseline_score)

In [ ]:
score = 10.7302
while True:
    feature_results = feature_selection(df_no_response, df_response, test_model, 
                                        starting_features=selected_features, 
                                        features=[f for f in all_features if f not in selected_features])
                                        # features=[f for f in all_features if f not in selected_features and any(item in f for item in ['temp_','irr_']) and not any(item in f for item in ['_corr','_median'])])
    new_ft = feature_results['feature'].iloc[0]
    new_score = feature_results['score'].iloc[0]
    score = new_score
    selected_features = selected_features + [new_ft]
    print(new_score, new_ft)
    print(selected_features, '\n')

In [ ]:
score = 10.5065
while True:
    feature_results = feature_ablation(df_no_response, df_response, test_model, selected_features, dnt_list=[])
    new_ft = feature_results['feature'].iloc[0]
    new_score = feature_results['score'].iloc[0]
    score = new_score
    selected_features = [f for f in selected_features if f != new_ft]
    print(new_score, new_ft)
    print(selected_features, '\n')
    # else:
    #     break

In [ ]:
print(len(selected_features))
print(selected_features)